In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import json
import requests
import time
from datetime import datetime
from google.cloud.logging import DESCENDING, Client
plt.rcParams['axes.grid'] = True
plt.rcParams['grid.alpha'] = 0.3
plt.rcParams['axes.titlesize'] = 18

In [2]:
client = Client(project='ultra-sound-relay')

/usr/local/lib/python3.8/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


In [63]:
payloads = []
for e in client.list_entries(filter_='jsonPayload.method="getPayload" AND "could not verify capella payload signature"'):
    payloads.append(e.to_api_repr()['jsonPayload'])
df = pd.DataFrame(payloads)
df= df.astype({'slot': 'int32'})
df.head()

,proposerPubkey,timestampRequestStart,timestampAfterDecode,method,mevBoostV,slot,slotStartSec,blockHash,version,contentLength,headSlot,msIntoSlot,idArg,service,error,level,msg,ua
0,0xa9c3eff1100a226df1f8277c72bc3971dcd304a5d9f2...,1.681339e+12,1.681339e+12,getPayload,v1.5.0,6209543,1.681339e+09,0xf592c8560d434c9a554f8b103c5c2c799b1faa3d0528...,,84890.0,6209542.0,1737.0,,relay/api,None,warning,could not verify capella payload signature,mev-boost/v1.5.0 Go-http-client/1.1
1,0xa9c3eff1100a226df1f8277c72bc3971dcd304a5d9f2...,1.681339e+12,1.681339e+12,getPayload,v1.5.0,6209543,1.681339e+09,0xf592c8560d434c9a554f8b103c5c2c799b1faa3d0528...,,84890.0,6209542.0,1903.0,,relay/api,None,warning,could not verify capella payload signature,mev-boost/v1.5.0 Go-http-client/1.1
2,0xa9c3eff1100a226df1f8277c72bc3971dcd304a5d9f2...,1.681339e+12,1.681339e+12,getPayload,v1.5.0,6209543,1.681339e+09,0xf592c8560d434c9a554f8b103c5c2c799b1faa3d0528...,,84890.0,6209542.0,1570.0,,relay/api,None,warning,could not verify capella payload signature,mev-boost/v1.5.0 Go-http-client/1.1
3,0xa9c3eff1100a226df1f8277c72bc3971dcd304a5d9f2...,1.681339e+12,1.681339e+12,getPayload,v1.5.0,6209543,1.681339e+09,0xf592c8560d434c9a554f8b103c5c2c799b1faa3d0528...,,84890.0,6209542.0,2234.0,,relay/api,None,warning,could not verify capella payload signature,mev-boost/v1.5.0 Go-http-client/1.1
4,0xa9c3eff1100a226df1f8277c72bc3971dcd304a5d9f2...,1.681339e+12,1.681339e+12,getPayload,v1.5.0,6209543,1.681339e+09,0xf592c8560d434c9a554f8b103c5c2c799b1faa3d0528...,,84890.0,6209542.0,2070.0,,relay/api,None,warning,could not verify capella payload signature,mev-boost/v1.5.0 Go-http-client/1.1


In [69]:
res = []
for s in np.sort(list(set(df['slot']))):
        time.sleep(0.1)
        url = 'http://57.128.92.65:3500/eth/v1/beacon/headers/{}'.format(s)
        headers = {'content-type': 'application/json'}
        r = requests.get(url, headers=headers)
        try: 
            code = r.json()['code']
            print(s, False)
            res.append((s, False))
        except KeyError:
            print(s, True)
            res.append((s, True))
res = np.asarray(res)
missed = np.asarray([v[0] for v in res if v[1] == 0])

6209543 False
6209609 False
6209620 True
6209643 False
6209655 False
6209723 False
6209731 False
6209875 False
6209877 False
6209885 True
6209902 False
6209920 False
6209957 True
6209967 False
6209986 False
6210036 False
6210037 True
6210054 False
6210089 False
6210103 False
6210214 False
6210402 True
6210534 True
6210726 True
6211358 True
6211845 True
6212101 True
6212166 True
6212625 True
6212769 True
6212967 True
6213249 True
6213853 True
6213855 True
6214241 True
6214321 True
6214767 True
6214776 True


## missed slots in general

In [16]:
pd.Timestamp.utcnow()

Timestamp('2023-04-13 15:35:26.669243+0000', tz='UTC')

In [26]:
fixTime = "2023-04-13T01:00:00.000000Z"
now = "2023-04-13T15:42:04.644881Z"
pdFilter = 'jsonPayload.method="getPayload" AND "getPayload request received" ' + \
           'AND timestamp>=' + '\"' + fixTime + '\" ' + \
           'AND timestamp<=' + '\"' + (now).strftime('%Y-%m-%dT%H:%M:%S.%fZ') + '\"'
pdFilter

'jsonPayload.method="getPayload" AND "getPayload request received" AND timestamp>="2023-04-13T01:00:00.000000Z" AND timestamp<="2023-04-13T15:42:04.644881Z"'

In [27]:
payloads = []
for e in client.list_entries(filter_=pdFilter):
    payloads.append(e.to_api_repr()['jsonPayload'])
df = pd.DataFrame(payloads)
df= df.astype({'slot': 'int32'})
df.head()

,timestampRequestStart,proposerPubkey,timestampAfterDecode,method,slotStartSec,slot,mevBoostV,timestampAfterSignatureVerify,blockHash,contentLength,headSlot,version,msIntoSlot,idArg,service,level,msg,ua
0,1.681348e+12,0x977fd52fb4ed4ac95f4e53f13c3b3fc1c75a262ddb26...,1.681348e+12,getPayload,1.681348e+09,6210301,-,1.681348e+12,0x0bb298a8461760e568a220e5c4a08fbc4a4a727e17fb...,85269.0,6210300.0,,892.0,,relay/api,info,getPayload request received,Go-http-client/1.1
1,1.681348e+12,0x8c6501411669dc668e55c3360224cc01a6045297a9d6...,1.681348e+12,getPayload,1.681348e+09,6210305,-,1.681348e+12,0xa2a7ec0c1c5613db87a13fd01f1529fc8f5b412adf8e...,91629.0,6210304.0,,691.0,,relay/api,info,getPayload request received,Go-http-client/1.1
2,1.681348e+12,0x8749f7985be5afb3b511b54f51d023b770aa39558d04...,1.681348e+12,getPayload,1.681348e+09,6210306,v1.5.0,1.681348e+12,0xa321568fa3c4ab3da88e58ad0526ce9ef9ff1713e5a0...,96112.0,6210305.0,,1029.0,,relay/api,info,getPayload request received,mev-boost/v1.5.0
3,1.681348e+12,0x8ae37af7b6b7034375f13e41aec4ade83520555de5af...,1.681348e+12,getPayload,1.681348e+09,6210307,-,1.681348e+12,0xf500b86e3189cbc311b0c97c1474e3901da6a1f76d5f...,69867.0,6210306.0,,626.0,,relay/api,info,getPayload request received,Go-http-client/1.1
4,1.681348e+12,0x95f3dcaf289d5fc35b0fa96fb9872f309235b149a7d4...,1.681348e+12,getPayload,1.681348e+09,6210309,v1.5.0,1.681348e+12,0xb6c18d0cc42cc9028be80612ce69b051961d02f14cd0...,60868.0,6210308.0,,1889.0,,relay/api,info,getPayload request received,mev-boost/v1.5.0


In [42]:
slots = np.sort(list(set(df.slot.values)))

In [44]:
res = []
for s in slots:
    time.sleep(0.1)
    url = 'http://57.128.92.65:3500/eth/v1/beacon/headers/{}'.format(s)
    headers = {'content-type': 'application/json'}
    r = requests.get(url, headers=headers)
    try: 
        code = r.json()['code']
        print(s, False)
        res.append((s, False))
    except KeyError:
        print(s, True)
        res.append((s, True))
res = np.asarray(res)
missed = np.asarray([v[0] for v in res if v[1] == 0])

5403404 True
5404228 True
5404288 True
5404599 True
5405124 True
5405499 True
5405719 True
5405843 True
5405954 True
5406366 True
5406692 True
6210301 True
6210305 True
6210306 True
6210307 True
6210309 True
6210314 True
6210330 True
6210338 True
6210350 True
6210352 True
6210363 True
6210366 True
6210368 True
6210369 True
6210370 True
6210372 True
6210376 True
6210381 True
6210383 True
6210397 True
6210402 True
6210405 True
6210409 True
6210436 True
6210441 True
6210442 True
6210443 True
6210446 True
6210450 True
6210469 True
6210480 True
6210500 True
6210501 True
6210505 True
6210507 True
6210511 True
6210517 True
6210519 True
6210532 True
6210533 True
6210534 True
6210544 True
6210550 True
6210557 True
6210560 True
6210582 True
6210588 True
6210589 True
6210599 True
6210614 True
6210619 True
6210620 True
6210629 True
6210632 True
6210641 True
6210643 True
6210645 True
6210646 True
6210660 True
6210663 True
6210664 True
6210670 True
6210671 True
6210686 True
6210696 True
6210700 True

6213484 True
6213488 True
6213489 True
6213492 True
6213505 True
6213506 True
6213512 True
6213513 True
6213521 True
6213523 True
6213534 True
6213537 True
6213538 True
6213542 True
6213544 True
6213550 True
6213554 True
6213556 True
6213573 True
6213581 True
6213592 True
6213595 True
6213599 True
6213603 True
6213604 True
6213606 True
6213607 True
6213610 True
6213615 True
6213618 True
6213621 True
6213645 True
6213646 True
6213647 True
6213650 True
6213654 True
6213656 True
6213659 True
6213661 True
6213666 True
6213673 True
6213675 True
6213680 True
6213681 True
6213683 True
6213691 True
6213692 True
6213699 True
6213706 True
6213715 True
6213732 True
6213739 True
6213744 True
6213753 True
6213754 True
6213759 True
6213760 True
6213763 True
6213766 True
6213768 False
6213770 True
6213784 True
6213786 True
6213790 True
6213791 True
6213792 True
6213793 True
6213796 True
6213807 True
6213812 True
6213819 True
6213824 True
6213825 True
6213826 True
6213827 True
6213830 True
6213837 Tru

In [45]:
missed

array([6212305, 6212576, 6213095, 6213260, 6213300, 6213768, 6214204],
      dtype=int32)

In [47]:
fixTime = "2023-04-13T01:00:00.000000Z"
now = "2023-04-13T15:42:04.644881Z"
pdFilter = 'jsonPayload.method="getPayload" AND "failed getting execution payload (2/2)" ' + \
           'AND timestamp>=' + '\"' + fixTime + '\" ' + \
           'AND timestamp<=' + '\"' + now + '\"'
pdFilter

'jsonPayload.method="getPayload" AND "failed getting execution payload (2/2)" AND timestamp>="2023-04-13T01:00:00.000000Z" AND timestamp<="2023-04-13T15:42:04.644881Z"'

In [51]:
notavail = []
for e in client.list_entries(filter_=pdFilter):
    notavail.append(e.to_api_repr()['jsonPayload'])
na = pd.DataFrame(notavail)
na = na.astype({'slot': 'int32'})
na.head()

,proposerPubkey,timestampRequestStart,timestampAfterDecode,timestampAfterAlreadyDelivered,method,blockHash,slot,slotStartSec,timestampAfterSignatureVerify,mevBoostV,headSlot,contentLength,version,msIntoSlot,idArg,service,level,msg,ua
0,0x8473b2c0336767a3aca2d80aeff088bbba9edfd59d4a...,1.681350e+12,1.681350e+12,1.681350e+12,getPayload,0xca0c0739f0fd6bc642bcb7954127684c9f10211914a4...,6210534,1.681350e+09,1.681350e+12,v1.5.0,6210533.0,96121.0,,1145.0,,relay/api,warning,failed getting execution payload (2/2),mev-boost/v1.5.0
1,0x8900823387854ddb557574d48c2bc4aa336eb1eb0fe8...,1.681359e+12,1.681359e+12,1.681359e+12,getPayload,0x2a9bfdcb0b322bb3b4879168d617c397a3123aadc978...,6211213,1.681359e+09,1.681359e+12,v1.5.0,6211212.0,89940.0,,1632.0,lido,relay/api,warning,failed getting execution payload (2/2),mev-boost/v1.5.0
2,0x8900823387854ddb557574d48c2bc4aa336eb1eb0fe8...,1.681359e+12,1.681359e+12,1.681359e+12,getPayload,0x2a9bfdcb0b322bb3b4879168d617c397a3123aadc978...,6211213,1.681359e+09,1.681359e+12,v1.5.0,6211212.0,89940.0,,1084.0,lido,relay/api,warning,failed getting execution payload (2/2),mev-boost/v1.5.0
3,0x8900823387854ddb557574d48c2bc4aa336eb1eb0fe8...,1.681359e+12,1.681359e+12,1.681359e+12,getPayload,0x2a9bfdcb0b322bb3b4879168d617c397a3123aadc978...,6211213,1.681359e+09,1.681359e+12,v1.5.0,6211212.0,89940.0,,1360.0,lido,relay/api,warning,failed getting execution payload (2/2),mev-boost/v1.5.0
4,0x8900823387854ddb557574d48c2bc4aa336eb1eb0fe8...,1.681359e+12,1.681359e+12,1.681359e+12,getPayload,0x2a9bfdcb0b322bb3b4879168d617c397a3123aadc978...,6211213,1.681359e+09,1.681359e+12,v1.5.0,6211212.0,89940.0,,1925.0,lido,relay/api,warning,failed getting execution payload (2/2),mev-boost/v1.5.0


In [58]:
actual = missed[~np.in1d(missed, na['slot'].values)]
actual

array([6214204], dtype=int32)

In [62]:
pruned = df.sort_values('msIntoSlot', ascending=True).drop_duplicates('slot').sort_index()
pruned[pruned['slot'].isin(actual)]['msIntoSlot']

944    5659.0
Name: msIntoSlot, dtype: float64

In [ ]:
6212305 we dont have payload
6212576 we dont have payload
6213095 we dont have payload
6213260 we dont have payload
6213300 we dont have payload
6213768 we dont have payload
slot 6214204 too late 5659ms into slot

# checking other relays

In [ ]:
# total_df = pd.DataFrame()
# while start > end:
#     print('start={}'.format(start))
#     payload['cursor'] = start
#     r = requests.get(
# #         'https://relay.ultrasound.money/relay/v1/data/bidtraces/proposer_payload_delivered',
# #         'https://agnostic-relay.net/relay/v1/data/bidtraces/proposer_payload_delivered',
#         'https://boost-relay.flashbots.net/relay/v1/data/bidtraces/proposer_payload_delivered',
#         params=payload
#     )
#     out_df = pd.DataFrame(json.loads(r.text))
#     total_df = total_df.append(out_df)
#     total_df = total_df.sort_values('slot').reset_index(drop=True)
#     start = int(total_df['slot'].min())
# len(total_df)

In [ ]:
#         'https://relay.ultrasound.money/relay/v1/data/bidtraces/proposer_payload_delivered',
#         'https://agnostic-relay.net/relay/v1/data/bidtraces/proposer_payload_delivered',


In [91]:
relayURLs = [
"https://boost-relay.flashbots.net",
"https://bloxroute.max-profit.blxrbdn.com",
"https://bloxroute.ethical.blxrbdn.com",
"https://bloxroute.regulated.blxrbdn.com",
"https://builder-relay-mainnet.blocknative.com",
"https://relay.edennetwork.io",
"https://relay.ultrasound.money",
"https://agnostic-relay.net",
"https://aestus.live"
]

In [106]:
slot=6215700
for url in relayURLs:
#     print('{}/relay/v1/data/bidtraces/builder_blocks_received?slot={}'.format(url, slot))
    r = requests.get(
        '{}/relay/v1/data/bidtraces/builder_blocks_received?slot={}'.format(url, slot),
    )
    res = json.loads(r.text)
    print(url)
    print(len(res))

https://boost-relay.flashbots.net
608
https://bloxroute.max-profit.blxrbdn.com
30
https://bloxroute.ethical.blxrbdn.com
0
https://bloxroute.regulated.blxrbdn.com
0
https://builder-relay-mainnet.blocknative.com
0
https://relay.edennetwork.io
0
https://relay.ultrasound.money
0
https://agnostic-relay.net
0
https://aestus.live
0


In [78]:
slot = 6215225
# url = 'https://relay.ultrasound.money/'
# url = 'https://agnostic-relay.net/'
url = 'https://boost-relay.flashbots.net/'
r = requests.get(
        '{}relay/v1/data/bidtraces/builder_blocks_received?slot={}'.format(url, slot),
)
res = json.loads(r.text)
len(res)

756

# full script

In [111]:
client = Client(project='ultra-sound-relay')

# construct query
now = pd.Timestamp.utcnow()
prev = now - pd.Timedelta('1d')
logFilter = 'jsonPayload.method="getPayload" AND "getPayload request received" ' + \
        'AND timestamp>=' + '\"' + (prev).strftime('%Y-%m-%dT%H:%M:%S.%fZ') + '\" ' + \
        'AND timestamp<=' + '\"' + (now).strftime('%Y-%m-%dT%H:%M:%S.%fZ') + '\"'

# read results into a df
payloads = []
for e in client.list_entries(filter_=logFilter):
    payloads.append(e.to_api_repr()['jsonPayload'])
payloads = pd.DataFrame(payloads)
payloads = payloads.astype({'slot': 'int32'})
slots = np.sort(list(set(payloads.slot.values)))

# check if block exists for that slot
res = []
for s in slots:
    time.sleep(0.1)
    url = 'http://57.128.92.65:3500/eth/v1/beacon/headers/{}'.format(s)
    headers = {'content-type': 'application/json'}
    r = requests.get(url, headers=headers)
    try: 
        code = r.json()['code']
        print(s, False)
        res.append((s, False))
    except KeyError:
        print(s, True)
        res.append((s, True))
res = np.asarray(res)
missed = np.asarray([v[0] for v in res if v[1] == 0])

# Ignore slots that we don't have the execution payload for.
noPayloadFilter = 'jsonPayload.method="getPayload" AND "failed getting execution payload (2/2)" ' + \
           'AND timestamp>=' + '\"' + (prev).strftime('%Y-%m-%dT%H:%M:%S.%fZ') + '\" ' + \
           'AND timestamp<=' + '\"' + (now).strftime('%Y-%m-%dT%H:%M:%S.%fZ') + '\"'
notavail = []
for e in client.list_entries(filter_=noPayloadFilter):
    notavail.append(e.to_api_repr()['jsonPayload'])
na = pd.DataFrame(notavail)
na = na.astype({'slot': 'int32'})
actual = missed[~np.in1d(missed, na['slot'].values)]



/usr/local/lib/python3.8/site-packages/google/auth/_default.py:78: UserWarning: Your application has authenticated using end user credentials from Google Cloud SDK without a quota project. You might receive a "quota exceeded" or "API not enabled" error. See the following page for troubleshooting: https://cloud.google.com/docs/authentication/adc-troubleshooting/user-creds. 
  warnings.warn(_CLOUD_SDK_CREDENTIALS_WARNING)


5407724 True
5407737 True
5408186 True
5409410 True
5409465 True
5409558 True
5409727 True
5409792 True
5410148 True
5410911 True
5411358 True
5411542 True
5412242 True
5412440 True
5412483 True
5412605 True
5413740 True
6214032 True
6214033 True
6214038 True
6214043 True
6214050 True
6214051 True
6214054 True
6214060 True
6214061 True
6214062 True
6214063 True
6214065 True
6214067 True
6214076 True
6214078 True
6214080 True
6214089 True
6214095 True
6214096 True
6214101 True
6214102 True
6214103 True
6214107 True
6214108 True
6214116 True
6214117 True
6214124 True
6214139 True
6214143 True
6214146 True
6214147 True
6214155 True
6214162 True
6214164 True
6214166 True
6214167 True
6214168 True
6214181 True
6214190 True
6214193 True
6214197 True
6214198 True
6214204 False
6214212 True
6214221 True
6214222 True
6214224 True
6214227 True
6214228 True
6214230 True
6214231 True
6214234 True
6214241 True
6214245 True
6214254 True
6214255 True
6214256 True
6214257 True
6214267 True
6214269 Tru

6216223 True
6216224 True
6216226 True
6216227 True
6216228 True
6216232 True
6216235 True
6216238 True
6216240 True
6216241 True
6216242 True
6216248 True
6216249 True
6216250 True
6216252 True
6216254 True
6216256 True
6216258 True
6216267 True
6216270 True
6216273 True
6216274 True
6216276 True
6216278 True
6216280 True
6216281 True
6216283 True
6216287 True
6216291 True
6216292 True
6216293 True
6216294 True
6216296 True
6216300 True
6216301 True
6216302 True
6216303 True
6216304 True
6216307 True
6216344 True
6216346 True
6216348 True
6216349 True
6216353 True
6216354 True
6216357 True
6216359 True
6216361 True
6216366 True
6216367 True
6216374 True
6216375 True
6216380 True
6216383 True
6216385 True
6216387 True
6216388 True
6216389 True
6216391 True
6216395 True
6216396 True
6216397 True
6216399 True
6216400 True
6216407 True
6216419 True
6216422 True
6216423 True
6216425 True
6216432 True
6216433 True
6216440 True
6216447 True
6216450 True
6216455 True
6216456 True
6216458 True

6218051 True
6218057 True
6218058 True
6218059 True
6218062 True
6218063 True
6218068 True
6218069 True
6218070 True
6218074 True
6218081 True
6218082 True
6218083 True
6218084 True
6218087 True
6218092 True
6218094 True
6218096 True
6218097 True
6218098 True
6218099 True
6218101 True
6218105 True
6218107 True
6218111 True
6218115 True
6218116 True
6218121 True
6218126 True
6218131 True
6218133 True
6218136 True
6218137 True
6218138 True
6218143 True
6218144 True
6218145 True
6218146 True
6218147 True
6218148 True
6218149 True
6218158 True
6218159 True
6218164 True
6218170 True
6218172 True
6218174 True
6218178 True
6218180 True
6218181 True
6218183 True
6218186 True
6218193 True
6218194 True
6218195 True
6218201 True
6218203 True
6218205 True
6218208 True
6218214 True
6218215 True
6218221 True
6218224 True
6218225 True
6218227 True
6218228 True
6218230 True
6218232 True
6218233 True
6218234 True
6218235 True
6218239 True
6218240 True
6218243 True
6218245 True
6218246 True
6218263 True

6219887 True
6219890 True
6219892 True
6219893 True
6219894 True
6219897 True
6219898 True
6219901 True
6219903 True
6219905 True
6219909 True
6219911 True
6219914 True
6219917 True
6219919 True
6219920 True
6219925 True
6219929 True
6219932 True
6219933 True
6219938 True
6219939 True
6219940 True
6219944 True
6219949 True
6219950 True
6219953 True
6219956 True
6219959 True
6219964 True
6219968 True
6219971 True
6219972 True
6219974 True
6219975 True
6219977 True
6219985 True
6219988 True
6219992 True
6219993 True
6219997 True
6219998 True
6220000 True
6220001 True
6220007 True
6220015 True
6220017 True
6220019 True
6220024 True
6220025 True
6220029 True
6220031 True
6220032 True
6220034 True
6220036 True
6220039 True
6220044 True
6220046 True
6220047 True
6220048 True
6220049 True
6220050 True
6220052 True
6220059 True
6220061 True
6220062 True
6220063 True
6220067 True
6220069 True
6220070 True
6220075 True
6220076 True
6220077 True
6220078 True
6220079 True
6220082 True
6220083 True

Series([], Name: msIntoSlot, dtype: float64)

In [114]:
missed

array([6214204, 6215236, 6215367, 6216729, 6220879], dtype=int32)

In [113]:
pruned = payloads.sort_values('msIntoSlot', ascending=True).drop_duplicates('slot').sort_index()
pruned[pruned['slot'].isin(actual)][['slot', 'msIntoSlot']]

,slot,msIntoSlot
42,6214204,5659.0
466,6215367,3673.0
2583,6220879,2110.0
